In [14]:
import pandas as pd
#import pandas_datareader as pdr
#import pandas_datareader.data as web
import pandas.plotting
import numpy as np
import matplotlib.pyplot as plt
# Turn interactive plotting off
plt.ioff()
#% matplotlib inline
plt.style.use('ggplot')
#import pandas.DateOffset
import datetime as dt
import time
import csv
import os
from scipy import stats



def list_of_stocks(my_stock_list):
    my_stocks=[]
    if (my_stock_list=='all'):
        Location_1=r'C:\Users\Sharyn\Desktop\Datasets\sandp500\SP500_list\All_SP500.csv'
    elif(my_stock_list=='low'):
        Location_1=r'C:\Users\Sharyn\Desktop\Datasets\sandp500\SP500_list\Low_SP500.csv'
        
    with open(Location_1,'r') as get_tickers:
        dayreader=csv.reader(get_tickers)
        #print (dayreader)
        for the_symbols in dayreader:
            my_stocks.append(the_symbols[0])     
    get_tickers.close
    
    return my_stocks


def get_stocks (ticker, startdate,enddate):
    location=r'C:\Users\Sharyn\Desktop\Datasets\sandp500\individual_stocks_5yr\{}_data.csv'.format(ticker)
    print(ticker)
    the_data=pd.read_csv(location,parse_dates=True,index_col='Date')
    #the_data.to_csv(r'C:\Users\Crystal\Desktop\datasets\stocks\{}_data.csv'.format(ticker.upper()))
    the_data=the_data[startdate:enddate]
    return the_data

def calculate_indicators (ticker, ticker_data):
    #print('I have the {} data.'.format(ticker))
    #print(ticker_data.head())
    
    ticker_data=the_slope(ticker_data)
    ticker_data['10ma']=ticker_data['Close'].rolling(window=10).mean()
    ticker_data.dropna(inplace=True)
    ticker_data['20ma']=ticker_data['Close'].rolling(window=20).mean()
    ticker_data['+2sigma']=ticker_data['20ma']+((ticker_data['Close'].rolling(window=20).std())*2)
    ticker_data['-2sigma']=ticker_data['20ma']-((ticker_data['Close'].rolling(window=20).std())*2)
    ticker_data['Sigma_Spread']=ticker_data['+2sigma']-ticker_data['-2sigma']
    ticker_data['HL_Spread']=ticker_data['High']-ticker_data['Low']
    ticker_data['LL_Spread']=ticker_data.Low.diff()
    ticker_data['Vol_mavg']=ticker_data['Volume'].rolling(window=20).mean()
    ticker_data['Vol_pct']=ticker_data['Volume'].pct_change()
    ticker_data['Daily_Return']=((ticker_data.Close-ticker_data.Close.shift(1))/ticker_data.Close.shift(1))*100
    ticker_data['Daily_Return_ma']=ticker_data['Daily_Return'].rolling(window=20).mean()
    the_rsi=rsi_function(ticker_data.Close)
    ticker_data['RSI']=the_rsi
    #trend_slope,m_rate=the_slope(ticker_data.Close)
    #ticker_data['Slope']=trend_slope
    #ticker_data['Slope_rate']=m_rate
    ticker_data.dropna(inplace=True)
    
    ticker_data.sort_index(ascending=True,inplace=True)
    
    #print(ticker_data.head())
    #print(ticker_data.info())
    return ticker_data

def store_indicated_data(ticker,indicated_data):
    Location=r'C:\Users\Sharyn\Desktop\Datasets\sandp500\indicated_data\{}_indicated.csv'.format(ticker)
    indicated_data.to_csv(Location)


def plot_the_stock(ticker,ticker_data,startdate):
    enddate=ticker_data.index[ticker_data.shape[0]-1].date()
    #print(ticker)
    #print(enddate)
    #print(startdate.date())
    #print(ticker_data.head())
    ##print(ticker_data)
    #ticker_data.sort_index(ascending=True,inplace=True)
        
    fig=plt.figure(figsize=(15,15))
    fig.suptitle('{} Plot'.format(ticker))
 
    ax=fig.add_subplot(411)
    ax1=fig.add_subplot(412)
    ax2=fig.add_subplot(413)
    ax3=fig.add_subplot(414)

    ax.set_xlabel('Date')
    ax.set_ylabel('Price')

    ax.set_title(label='{} Stock Data'.format(ticker),loc='left')
##    
    #ax.plot(ticker_data[startdate:enddate][['Close','10ma','20ma']])
    ax.plot(ticker_data[startdate:enddate]['Close'],color='green')
    ax.plot(ticker_data[startdate:enddate]['10ma'],color='yellow')
    ax.plot(ticker_data[startdate:enddate]['20ma'],color='red')
    ax.plot(ticker_data[startdate:enddate][['+2sigma','-2sigma']],color='black')
    #ax.plot(ticker_data[startdate:enddate][['High','Low']],color='g')
##    
    ax.legend(ticker_data[['Close','10ma','20ma','+2sigma','-2sigma']])
    
## 
    ax1.plot(ticker_data[startdate:enddate]['RSI'],color='blue')
    #ax1.ylim(0,100)
    ax1.axhline(y=70)
    ax1.axhline(y=20)
    
##    
    ax2.plot(ticker_data[startdate:enddate]['Vol_mavg'])
    ax2.bar(ticker_data[startdate:enddate].index,ticker_data[startdate:enddate]['Volume'],color='black')
##    
    ax3.set_ylabel('% Daily Return ')
    ax3.bar(ticker_data[startdate:enddate].index,ticker_data[startdate:enddate]['Daily_Return'],color='black')
    ax3.plot(ticker_data[startdate:enddate]['Daily_Return_ma'],color='r')
    ax3.legend()
    
 
        
    Location=(r'C:\Users\Sharyn\Desktop\Datasets\sandp500\indicated_data\{}_plot.pdf'.format(ticker))
    plt.savefig(Location)
        
        
    
#-------------------------------------------------------------------------------------


In [15]:
def p10(ticker,ticker_data,target_pct,stop_loss_pct,num_shares,enum,do_plots):
    
    L=len(ticker_data.index)
    in_trade=0
    count_target=0
    count_loss=0
    count_trade=0
    make_money=0
    lose_money=0
    target_list=[]
    loss_list=[]
    trade_date_list=[]
    ticker_data['Entry_Date']=np.nan
    ticker_data['Exit_Date']=np.nan
    ticker_data['Duration']=np.nan
    ticker_data['Target']=np.nan
    ticker_data['Stop_Loss']=np.nan
    ticker_data['GL']=np.nan
    ticker_data['GL_Status']=np.nan
    scenario='P10'
    
    
    
    #print(L)
    for x in range(L):
        ma_check=ticker_data['Close'].iloc[x]-ticker_data['10ma'].iloc[x]
        daily_return_ma=ticker_data['Daily_Return_ma'].iloc[x]
        #print(ma_check)
        #ma_check_previous= a['Close'].iloc[x-1]-a['10ma'].iloc[x-1]
        if (x==0):
            #continue
            ma_check_previous=ticker_data['Close'].iloc[0]-ticker_data['10ma'].iloc[0]
        else:
            ma_check_previous= ticker_data['Close'].iloc[x-1]-ticker_data['10ma'].iloc[x-1]
    

        #moving average uptrend transition--------------------------------------------------
        if (ma_check>0) and (ma_check_previous<0) and (daily_return_ma>0) and in_trade==0:
            in_trade=1
            count_trade=count_trade+1
            #print(ticker_data.index[x],'----',ticker_data['Close'].iloc[x])
            ticker_data.Target.iloc[x]=ticker_data.Close.iloc[x]+(ticker_data.Close.iloc[x]*target_pct)
            #ticker_data['Target']=ticker_data.Close.iloc[x]+(ticker_data.Close.iloc[x]*target_pct)
            ticker_data.Stop_Loss.iloc[x]=ticker_data.Close.iloc[x]-(ticker_data.Close.iloc[x]*stop_loss_pct)
            #ticker_data['Stop_Loss']=ticker_data.Close.iloc[x]-(ticker_data.Close.iloc[x]*stop_loss_pct)
            my_target=ticker_data['Target'].iloc[x]
            my_loss=ticker_data['Stop_Loss'].iloc[x]
            the_gain=(my_target-ticker_data['Close'].iloc[x])*num_shares
            the_loss=(my_loss-ticker_data['Close'].iloc[x])*num_shares
            target_list.append(ticker_data['Target'].iloc[x])
            loss_list.append(ticker_data['Stop_Loss'].iloc[x])
            trade_date_list.append(ticker_data.index[x])
            x_on_entry=x
            #ticker_data['Symbol'].iloc[x]=ticker
            #print(ticker_data['Target'].iloc[x],'---',ticker_data['Stop_Loss'].iloc[x],'---',count_trade)
            #print(ticker_data['High'].iloc[x],'---',ticker_data['Low'].iloc[x])
            continue
            
        if (in_trade==1):
            
            if (my_target<ticker_data['High'].iloc[x]):
                #print('Target Reached:',ticker_data.index[x],'---',ticker_data['Target'].iloc[x_on_entry],'---',ticker_data['High'].iloc[x])
                count_target=count_target+1
                make_money=the_gain+make_money
                ticker_data.Entry_Date=ticker_data.index[x_on_entry].date()
                ticker_data.Exit_Date=ticker_data.index[x].date()
                #ticker_data.Duration=ticker_data.Exit_Date.iloc[x]-ticker_data.Entry_Date
                the_duration=ticker_data.Exit_Date.iloc[x]-ticker_data.Entry_Date
                ticker_data.Duration=the_duration.dt.days
                ticker_data.Target=ticker_data.Close.iloc[x_on_entry]+(ticker_data.Close.iloc[x_on_entry]*target_pct)
                ticker_data.Stop_Loss=ticker_data.Close.iloc[x_on_entry]-(ticker_data.Close.iloc[x_on_entry]*stop_loss_pct)
                ticker_data.GL=the_gain
                ticker_data.GL_Status=1
                store_trade_data(ticker,ticker_data,ticker_data.iloc[x_on_entry],scenario,enum)
                in_trade=0
            elif(my_loss>ticker_data['Low'].iloc[x]):
                #print('Stop loss:',ticker_data.index[x],'---',ticker_data['Stop_Loss'].iloc[x_on_entry],'---',ticker_data['Low'].iloc[x])
                count_loss=count_loss+1
                lose_money=the_loss+lose_money
                ticker_data.Entry_Date=ticker_data.index[x_on_entry].date()
                ticker_data.Exit_Date=ticker_data.index[x].date()
                #ticker_data.Duration=ticker_data.Exit_Date.iloc[x]-ticker_data.Entry_Date
                the_duration=ticker_data.Exit_Date.iloc[x]-ticker_data.Entry_Date
                ticker_data.Duration=the_duration.dt.days
                ticker_data.Target=ticker_data.Close.iloc[x_on_entry]+(ticker_data.Close.iloc[x_on_entry]*target_pct)
                ticker_data.Stop_Loss=ticker_data.Close.iloc[x_on_entry]-(ticker_data.Close.iloc[x_on_entry]*stop_loss_pct)
                ticker_data.GL=the_loss
                ticker_data.GL_Status=0
                store_trade_data(ticker,ticker_data,ticker_data.iloc[x_on_entry],scenario,enum)
                in_trade=0
    
    net_amount=make_money+lose_money
    pct_trade_gain=count_target/(count_target+count_loss)
    
    if (do_plots=='yes'):
        print('Target pct: {}'.format(target_pct))
        print('Stop Loss pct: {}'.format(stop_loss_pct))
        print('Total times in trade: {}'.format(count_trade))
        print('Number of gains: {}'.format(count_target))
        print('Number of losses: {}'.format(count_loss))
        print('% of trades that were gains: {:.2f}'.format(pct_trade_gain))
        print('Total gains:{:.2f}'.format(make_money))
        print('Total_losses: {:.2f}'.format(lose_money))
        print('net amount in the P10 scenerio: {:.2f}'.format(net_amount))
        print('-----------------------------------------------------------\n')
        #print('Ticker_data:\n', ticker_data.head(2))
        #print(ticker_data.columns)
        trade_plot(ticker,ticker_data,target_list,loss_list,trade_date_list,scenario)
    #store_trade_data(ticker,ticker_data.iloc[x])
    

In [16]:
def trade_plot(ticker,ticker_data,target_list,loss_list,trade_date_list,scenario,entry_price_list,exit_price_list,exit_date_list,startdate=dt.datetime(2012,1,1)):
    enddate=ticker_data.index[ticker_data.shape[0]-1].date()
    
    the_df={'Date':trade_date_list,'Target':target_list,'Stop_Loss':loss_list,'Entry_price':entry_price_list,'Exit_price':exit_price_list,'Exit_date':exit_date_list}
    df=pd.DataFrame(data=the_df,columns=['Date','Target','Stop_Loss','Entry_price','Exit_price','Exit_date'])
    
    fig=plt.figure(figsize=(15,15))
    fig.suptitle('{} Trade Plot'.format(ticker))
 
    ax=fig.add_subplot(111)
    #ax1=fig.add_subplot(412)
    #ax2=fig.add_subplot(312)
    #ax3=fig.add_subplot(313)

    ax.set_xlabel('Date')
    ax.set_ylabel('Price')
    ax.set_title(label='{} Trade Stock Data'.format(ticker),loc='left')
    ax.plot(ticker_data[startdate:enddate]['Close'],color='green')
    ax.plot(ticker_data[startdate:enddate]['10ma'],color='yellow')
    ax.plot(ticker_data[startdate:enddate]['20ma'],color='red')
    ax.plot(ticker_data[startdate:enddate][['+2sigma','-2sigma']],color='black')
    ax.plot(ticker_data[startdate:enddate][['High','Low']],color='blue',linewidth=.5)
    ax.plot(df['Date'],df['Target'],'^',color='green')
    ax.plot(df['Date'],df['Stop_Loss'],'v',color='red')
    ax.plot(df['Date'],df['Entry_price'],'o',color='white')
    ax.plot(df['Exit_date'],df['Exit_price'],'o',color='black')
    ax.plot(ticker_data[startdate:enddate]['Close'],'o',color='green',markersize=2)

    
##    
    ax.legend(ticker_data[['Close','10ma','20ma','+2sigma','-2sigma','High','Low']])
    
    
    
    Location=(r'C:\Users\Sharyn\Desktop\Datasets\sandp500\{}_data\{}_{}_trade_plot.pdf'.format(scenario,ticker,scenario))
    plt.savefig(Location)
    

In [17]:
def store_trade_data(ticker,ticker_data,the_data,scenario,enum):
    Location=(r'C:\Users\Sharyn\Desktop\Datasets\sandp500\{}_data\{}_{}.csv'.format(scenario,ticker,scenario))
    if os.path.isfile(Location):
        with open(Location, 'a',newline='') as my_file:
            writer=csv.writer(my_file)
            writer.writerow(the_data)
            #ticker_data.to_csv(my_file)
    else:
        the_header=ticker_data.columns
        with open(Location, 'a',newline='') as my_file:
            headwriter=csv.writer(my_file)
            headwriter.writerow(the_header.values)
        with open(Location, 'a',newline='') as my_file:
            writer=csv.writer(my_file)
            writer.writerow(the_data)
            #ticker_data.to_csv(my_file)
    
    Location=(r'C:\Users\Sharyn\Desktop\Datasets\sandp500\{}_data\consolidated_{}_{}.csv'.format(scenario,scenario,enum))
    if os.path.isfile(Location):
        with open(Location, 'a',newline='') as my_file:
            writer=csv.writer(my_file)
            writer.writerow(the_data)
            #ticker_data.to_csv(my_file)
    else:
        the_header=ticker_data.columns
        with open(Location, 'a',newline='') as my_file:
            headwriter=csv.writer(my_file)
            headwriter.writerow(the_header.values)
        with open(Location, 'a',newline='') as my_file:
            writer=csv.writer(my_file)
            writer.writerow(the_data)
            #ticker_data.to_csv(my_file)
    
    

In [18]:
def TOT(ticker,ticker_data,target_pct,stop_loss_pct,num_shares,enum,do_plots):
    L=len(ticker_data.index)
    in_trade=0
    count_target=0
    count_loss=0
    count_trade=0
    make_money=0
    lose_money=0
    target_list=[]
    loss_list=[]
    trade_date_list=[]
    ticker_data['Entry_Date']=np.nan
    ticker_data['Exit_Date']=np.nan
    ticker_data['Duration']=np.nan
    ticker_data['Target']=np.nan
    ticker_data['Stop_Loss']=np.nan
    ticker_data['GL']=np.nan
    ticker_data['GL_Status']=np.nan
    scenario='TOT'
    
    for x in range(L):
        tot_check=ticker_data['10ma'].iloc[x]-ticker_data['20ma'].iloc[x]
        daily_return_ma=ticker_data['Daily_Return_ma'].iloc[x]
        
        #print(ma_check)
        
        if (x==0):
            #continue
            tot_check_previous=ticker_data['10ma'].iloc[0]-ticker_data['20ma'].iloc[0]
        else:
            tot_check_previous= ticker_data['10ma'].iloc[x-1]-ticker_data['20ma'].iloc[x-1]
    
    
        #TOT uptrend transition--------------------------------------------------
        if (tot_check>0) and (tot_check_previous<0) and (daily_return_ma>0) and in_trade==0:
            in_trade=1
            count_trade=count_trade+1
            #print(ticker_data.index[x],'----',ticker_data['Close'].iloc[x])
            ticker_data.Target.iloc[x]=ticker_data.Close.iloc[x]+(ticker_data.Close.iloc[x]*target_pct)
            #ticker_data['Target']=ticker_data.Close.iloc[x]+(ticker_data.Close.iloc[x]*target_pct)
            ticker_data.Stop_Loss.iloc[x]=ticker_data.Close.iloc[x]-(ticker_data.Close.iloc[x]*stop_loss_pct)
            #ticker_data['Stop_Loss']=ticker_data.Close.iloc[x]-(ticker_data.Close.iloc[x]*stop_loss_pct)
            my_target=ticker_data['Target'].iloc[x]
            my_loss=ticker_data['Stop_Loss'].iloc[x]
            the_gain=(my_target-ticker_data['Close'].iloc[x])*num_shares
            the_loss=(my_loss-ticker_data['Close'].iloc[x])*num_shares
            target_list.append(ticker_data['Target'].iloc[x])
            loss_list.append(ticker_data['Stop_Loss'].iloc[x])
            trade_date_list.append(ticker_data.index[x])
            x_on_entry=x
            #ticker_data['Symbol'].iloc[x]=ticker
            #print(ticker_data['Target'].iloc[x],'---',ticker_data['Stop_Loss'].iloc[x],'---',count_trade)
            #print(ticker_data['High'].iloc[x],'---',ticker_data['Low'].iloc[x])
            continue
            
        if (in_trade==1):
            
            if (my_target<ticker_data['High'].iloc[x]):
                #print('Target Reached:',ticker_data.index[x],'---',ticker_data['Target'].iloc[x_on_entry],'---',ticker_data['High'].iloc[x])
                count_target=count_target+1
                make_money=the_gain+make_money
                ticker_data.Entry_Date=ticker_data.index[x_on_entry].date()
                ticker_data.Exit_Date=ticker_data.index[x].date()
                #ticker_data.Duration=ticker_data.Exit_Date.iloc[x]-ticker_data.Entry_Date
                the_duration=ticker_data.Exit_Date.iloc[x]-ticker_data.Entry_Date
                ticker_data.Duration=the_duration.dt.days
                ticker_data.Target=ticker_data.Close.iloc[x_on_entry]+(ticker_data.Close.iloc[x_on_entry]*target_pct)
                ticker_data.Stop_Loss=ticker_data.Close.iloc[x_on_entry]-(ticker_data.Close.iloc[x_on_entry]*stop_loss_pct)
                ticker_data.GL=the_gain
                ticker_data.GL_Status=1
                store_trade_data(ticker,ticker_data,ticker_data.iloc[x_on_entry],scenario,enum)
                in_trade=0
            elif(my_loss>ticker_data['Low'].iloc[x]):
                #print('Stop loss:',ticker_data.index[x],'---',ticker_data['Stop_Loss'].iloc[x_on_entry],'---',ticker_data['Low'].iloc[x])
                count_loss=count_loss+1
                lose_money=the_loss+lose_money
                ticker_data.Entry_Date=ticker_data.index[x_on_entry].date()
                ticker_data.Exit_Date=ticker_data.index[x].date()
                #ticker_data.Duration=ticker_data.Exit_Date.iloc[x]-ticker_data.Entry_Date
                the_duration=ticker_data.Exit_Date.iloc[x]-ticker_data.Entry_Date
                ticker_data.Duration=the_duration.dt.days
                ticker_data.Target=ticker_data.Close.iloc[x_on_entry]+(ticker_data.Close.iloc[x_on_entry]*target_pct)
                ticker_data.Stop_Loss=ticker_data.Close.iloc[x_on_entry]-(ticker_data.Close.iloc[x_on_entry]*stop_loss_pct)
                ticker_data.GL=the_loss
                ticker_data.GL_Status=0
                store_trade_data(ticker,ticker_data,ticker_data.iloc[x_on_entry],scenario,enum)
                in_trade=0
                
    net_amount=make_money+lose_money
    pct_trade_gain=count_target/(count_target+count_loss)
    
    if (do_plots=='yes'):
        print('Target pct: {}'.format(target_pct))
        print('Stop Loss pct: {}'.format(stop_loss_pct))
        print('Total times in trade: {}'.format(count_trade))
        print('Number of gains: {}'.format(count_target))
        print('Number of losses: {}'.format(count_loss))
        print('% of trades that were gains: {:.2f}'.format(pct_trade_gain))
        print('Total gains:{:.2f}'.format(make_money))
        print('Total_losses: {:.2f}'.format(lose_money))
        print('net amount in the TOT scenerio: {:.2f}'.format(net_amount))
        print('-----------------------------------------------------------\n')
        #print('Ticker_data:\n', ticker_data.head(2))
        #print(ticker_data.columns)
        trade_plot(ticker,ticker_data,target_list,loss_list,trade_date_list,scenario)
    #store_trade_data(ticker,ticker_data.iloc[x])
                
                
                
                

In [19]:
def low_sigma(ticker,ticker_data,target_pct,stop_loss_pct,num_shares,enum,do_plots):
    L=len(ticker_data.index)
    in_trade=0
    count_target=0
    count_loss=0
    count_trade=0
    make_money=0
    lose_money=0
    target_list=[]
    loss_list=[]
    trade_date_list=[]
    ticker_data['Entry_Date']=np.nan
    ticker_data['Exit_Date']=np.nan
    ticker_data['Duration']=np.nan
    ticker_data['Target']=np.nan
    ticker_data['Stop_Loss']=np.nan
    ticker_data['GL']=np.nan
    ticker_data['GL_Status']=np.nan
    scenario='low_sigma'
    
    for x in range(L):
        sig_check=ticker_data['Close'].iloc[x]-ticker_data['-2sigma'].iloc[x]
        daily_return_ma=ticker_data['Daily_Return_ma'].iloc[x]
        
        #print(ma_check)
        
        if (x==0):
            #continue
            sig_check_previous=ticker_data['Close'].iloc[0]-ticker_data['-2sigma'].iloc[0]
        else:
            sig_check_previous= ticker_data['Close'].iloc[x-1]-ticker_data['-2sigma'].iloc[x-1]
    
    
        #Close below -2 sigma transistion--------------------------------------------------
        if (sig_check<0) and (sig_check_previous>0) and (daily_return_ma<0) and in_trade==0:
            in_trade=1
            count_trade=count_trade+1
            #print(ticker_data.index[x],'----',ticker_data['Close'].iloc[x])
            ticker_data.Target.iloc[x]=ticker_data.Close.iloc[x]+(ticker_data.Close.iloc[x]*target_pct)
            #ticker_data['Target']=ticker_data.Close.iloc[x]+(ticker_data.Close.iloc[x]*target_pct)
            ticker_data.Stop_Loss.iloc[x]=ticker_data.Close.iloc[x]-(ticker_data.Close.iloc[x]*stop_loss_pct)
            #ticker_data['Stop_Loss']=ticker_data.Close.iloc[x]-(ticker_data.Close.iloc[x]*stop_loss_pct)
            my_target=ticker_data['Target'].iloc[x]
            my_loss=ticker_data['Stop_Loss'].iloc[x]
            the_gain=(my_target-ticker_data['Close'].iloc[x])*num_shares
            the_loss=(my_loss-ticker_data['Close'].iloc[x])*num_shares
            target_list.append(ticker_data['Target'].iloc[x])
            loss_list.append(ticker_data['Stop_Loss'].iloc[x])
            trade_date_list.append(ticker_data.index[x])
            x_on_entry=x
            #ticker_data['Symbol'].iloc[x]=ticker
            #print(ticker_data['Target'].iloc[x],'---',ticker_data['Stop_Loss'].iloc[x],'---',count_trade)
            #print(ticker_data['High'].iloc[x],'---',ticker_data['Low'].iloc[x])
            continue
            
        if (in_trade==1):
            
            if (my_target<ticker_data['High'].iloc[x]):
                #print('Target Reached:',ticker_data.index[x],'---',ticker_data['Target'].iloc[x_on_entry],'---',ticker_data['High'].iloc[x])
                count_target=count_target+1
                make_money=the_gain+make_money
                ticker_data.Entry_Date=ticker_data.index[x_on_entry].date()
                ticker_data.Exit_Date=ticker_data.index[x].date()
                #ticker_data.Duration=ticker_data.Exit_Date.iloc[x]-ticker_data.Entry_Date
                the_duration=ticker_data.Exit_Date.iloc[x]-ticker_data.Entry_Date
                ticker_data.Duration=the_duration.dt.days
                ticker_data.Target=ticker_data.Close.iloc[x_on_entry]+(ticker_data.Close.iloc[x_on_entry]*target_pct)
                ticker_data.Stop_Loss=ticker_data.Close.iloc[x_on_entry]-(ticker_data.Close.iloc[x_on_entry]*stop_loss_pct)
                ticker_data.GL=the_gain
                ticker_data.GL_Status=1
                store_trade_data(ticker,ticker_data,ticker_data.iloc[x_on_entry],scenario,enum)
                in_trade=0
            elif(my_loss>ticker_data['Low'].iloc[x]):
                #print('Stop loss:',ticker_data.index[x],'---',ticker_data['Stop_Loss'].iloc[x_on_entry],'---',ticker_data['Low'].iloc[x])
                count_loss=count_loss+1
                lose_money=the_loss+lose_money
                ticker_data.Entry_Date=ticker_data.index[x_on_entry].date()
                ticker_data.Exit_Date=ticker_data.index[x].date()
                #ticker_data.Duration=ticker_data.Exit_Date.iloc[x]-ticker_data.Entry_Date
                the_duration=ticker_data.Exit_Date.iloc[x]-ticker_data.Entry_Date
                ticker_data.Duration=the_duration.dt.days
                ticker_data.Target=ticker_data.Close.iloc[x_on_entry]+(ticker_data.Close.iloc[x_on_entry]*target_pct)
                ticker_data.Stop_Loss=ticker_data.Close.iloc[x_on_entry]-(ticker_data.Close.iloc[x_on_entry]*stop_loss_pct)
                ticker_data.GL=the_loss
                ticker_data.GL_Status=0
                store_trade_data(ticker,ticker_data,ticker_data.iloc[x_on_entry],scenario,enum)
                in_trade=0
                
    net_amount=make_money+lose_money
    pct_trade_gain=count_target/(count_target+count_loss)
    
    if (do_plots=='yes'):
        print('Target pct: {}'.format(target_pct))
        print('Stop Loss pct: {}'.format(stop_loss_pct))
        print('Total times in trade: {}'.format(count_trade))
        print('Number of gains: {}'.format(count_target))
        print('Number of losses: {}'.format(count_loss))
        print('% of trades that were gains: {:.2f}'.format(pct_trade_gain))
        print('Total gains:{:.2f}'.format(make_money))
        print('Total_losses: {:.2f}'.format(lose_money))
        print('net amount in the low sigma scenerio: {:.2f}'.format(net_amount))
        print('-----------------------------------------------------------\n')
        #print('Ticker_data:\n', ticker_data.head(2))
        #print(ticker_data.columns)
        trade_plot(ticker,ticker_data,target_list,loss_list,trade_date_list,scenario)
    #store_trade_data(ticker,ticker_data.iloc[x])
                
                

In [20]:
def consolidated_summary(enum,my_target_pct,allowable_loss,trade_scenario):
    results_location=(r'C:\Users\Sharyn\Desktop\Datasets\sandp500\{}_data\consolidated_{}_{}.csv'.format(trade_scenario,trade_scenario,enum))
    cdf=pd.read_csv(results_location)
    print('Target pct:',my_target_pct)
    print('Stop Loss pct:',allowable_loss)
    print('Net G/L:',cdf.GL.sum())
    print('G/L pct:\n', cdf.GL_Status.value_counts(normalize=True))
    print('-------------------------------------------------------------')

In [21]:
def rsi_function(Close,n=14):
    deltas=np.diff(Close)
    seed=deltas[:n+1]
    up=seed[seed>=0].sum()/n
    down=-seed[seed<0].sum()/n
    
    rs=up/down
    rsi=np.zeros_like(Close)
    
    rsi[:n]=100.-100./(1.+rs)
    
    for i in range(n,len(Close)):
        delta=deltas[i-1]
        if delta >0:
            upval=delta
            downval=0.
        else:
            upval=0.
            downval=-delta
            
        up=(up*(n-1)+upval)/n
        down=(down*(n-1)+downval)/n
        
        rs=up/down
        rsi[i]=100.-100./(1.+rs)
        
    the_rsi=pd.DataFrame(rsi)
    rsi.reshape(the_rsi.shape[0],1)
    return rsi
        
    
    #print(seed,up,down,rs)

In [22]:
def the_slope(my_data):
    n=0
    p=5
    my_slope=[]
    slope_rate=[]
    
    my_data['Slope']=0
    my_data['Slope_rate']=0
   
    for i in range(0,len(my_data)):
        #print (i)
        if (i>5):
            y=my_data.Close.values
            y=y[n:p]
            x=np.arange(y.shape[0])
            #print(i,x,y)
            m,b,r_value,p_value,std_err=stats.linregress(x,y)
            my_slope.append(m)
            s_rate=my_slope[i]-my_slope[i-1]
            #print ('slope:',m)
            #print ('slope_rate:',s_rate)
            slope_rate.append(s_rate)
            my_data['Slope'].iloc[i]=m
            my_data['Slope_rate'].iloc[i]=s_rate
            
            #plt.grid()
            #plt.plot(x,y,color='green')
            #plt.scatter(x,y, color='green')
            #plt.plot(x,m*x+b,color='black')
            #plt.show()
            
            n=n+1
            p=p+1
            
        else:
            my_slope.append(0)
            slope_rate.append(0)
            
            my_data['Slope'].iloc[i]=0
            my_data['Slope_rate'].iloc[i]=0
            
        
    return my_data
        

In [23]:
def XXX(ticker,ticker_data,target_pct,stop_loss_pct,num_shares,enum,do_plots):
    
    L=len(ticker_data.index)
    in_trade=0
    count_target=0
    count_loss=0
    count_trade=0
    make_money=0
    lose_money=0
    target_list=[]
    loss_list=[]
    trade_date_list=[]
    ticker_data['Entry_Date']=np.nan
    ticker_data['Exit_Date']=np.nan
    ticker_data['Duration']=np.nan
    ticker_data['Target']=np.nan
    ticker_data['Stop_Loss']=np.nan
    ticker_data['GL']=np.nan
    ticker_data['GL_Status']=np.nan
    scenario='XXX'
    
    
    
    #print(L)
    for x in range(L):
        ma_check=ticker_data['Low'].iloc[x]-ticker_data['10ma'].iloc[x]
        daily_return_ma=ticker_data['Daily_Return_ma'].iloc[x]
        LL_Spread=ticker_data['LL_Spread'].iloc[x]
        Vol_pct=ticker_data['Vol_pct'].iloc[x]
        RSI=ticker_data['RSI'].iloc[x]
        Slope=ticker_data['Slope'].iloc[x]
        
        #print(ma_check)
        #ma_check_previous= a['Close'].iloc[x-1]-a['10ma'].iloc[x-1]
        if (x==0):
            #continue
            ma_check_previous=ticker_data['Low'].iloc[0]-ticker_data['10ma'].iloc[0]
        else:
            ma_check_previous= ticker_data['Low'].iloc[x-1]-ticker_data['10ma'].iloc[x-1]
    

        #trigger condition that gets you in the trade--------------------------------------------------
        if (ma_check>0) and (ma_check_previous<0) and (Slope>0.05) and in_trade==0:
            in_trade=1
            count_trade=count_trade+1
            #print(ticker_data.index[x],'----',ticker_data['Close'].iloc[x])
            ticker_data.Target.iloc[x]=ticker_data.Close.iloc[x]+(ticker_data.Close.iloc[x]*target_pct)
            #ticker_data['Target']=ticker_data.Close.iloc[x]+(ticker_data.Close.iloc[x]*target_pct)
            ticker_data.Stop_Loss.iloc[x]=ticker_data.Close.iloc[x]-(ticker_data.Close.iloc[x]*stop_loss_pct)
            #ticker_data['Stop_Loss']=ticker_data.Close.iloc[x]-(ticker_data.Close.iloc[x]*stop_loss_pct)
            my_target=ticker_data['Target'].iloc[x]
            my_loss=ticker_data['Stop_Loss'].iloc[x]
            the_gain=(my_target-ticker_data['Close'].iloc[x])*num_shares
            the_loss=(my_loss-ticker_data['Close'].iloc[x])*num_shares
            target_list.append(ticker_data['Target'].iloc[x])
            loss_list.append(ticker_data['Stop_Loss'].iloc[x])
            trade_date_list.append(ticker_data.index[x])
            x_on_entry=x
            #ticker_data['Symbol'].iloc[x]=ticker
            #print(ticker_data['Target'].iloc[x],'---',ticker_data['Stop_Loss'].iloc[x],'---',count_trade)
            #print(ticker_data['High'].iloc[x],'---',ticker_data['Low'].iloc[x])
            continue
            
        if (in_trade==1):
            
            if (my_target<ticker_data['High'].iloc[x]):
                #print('Target Reached:',ticker_data.index[x],'---',ticker_data['Target'].iloc[x_on_entry],'---',ticker_data['High'].iloc[x])
                count_target=count_target+1
                make_money=the_gain+make_money
                ticker_data.Entry_Date=ticker_data.index[x_on_entry].date()
                ticker_data.Exit_Date=ticker_data.index[x].date()
                #ticker_data.Duration=ticker_data.Exit_Date.iloc[x]-ticker_data.Entry_Date
                the_duration=ticker_data.Exit_Date.iloc[x]-ticker_data.Entry_Date
                ticker_data.Duration=the_duration.dt.days
                ticker_data.Target=ticker_data.Close.iloc[x_on_entry]+(ticker_data.Close.iloc[x_on_entry]*target_pct)
                ticker_data.Stop_Loss=ticker_data.Close.iloc[x_on_entry]-(ticker_data.Close.iloc[x_on_entry]*stop_loss_pct)
                ticker_data.GL=the_gain
                ticker_data.GL_Status=1
                store_trade_data(ticker,ticker_data,ticker_data.iloc[x_on_entry],scenario,enum)
                in_trade=0
            elif(my_loss>ticker_data['Low'].iloc[x]):
                #print('Stop loss:',ticker_data.index[x],'---',ticker_data['Stop_Loss'].iloc[x_on_entry],'---',ticker_data['Low'].iloc[x])
                count_loss=count_loss+1
                lose_money=the_loss+lose_money
                ticker_data.Entry_Date=ticker_data.index[x_on_entry].date()
                ticker_data.Exit_Date=ticker_data.index[x].date()
                #ticker_data.Duration=ticker_data.Exit_Date.iloc[x]-ticker_data.Entry_Date
                the_duration=ticker_data.Exit_Date.iloc[x]-ticker_data.Entry_Date
                ticker_data.Duration=the_duration.dt.days
                ticker_data.Target=ticker_data.Close.iloc[x_on_entry]+(ticker_data.Close.iloc[x_on_entry]*target_pct)
                ticker_data.Stop_Loss=ticker_data.Close.iloc[x_on_entry]-(ticker_data.Close.iloc[x_on_entry]*stop_loss_pct)
                ticker_data.GL=the_loss
                ticker_data.GL_Status=0
                store_trade_data(ticker,ticker_data,ticker_data.iloc[x_on_entry],scenario,enum)
                in_trade=0
    
    net_amount=make_money+lose_money
    pct_trade_gain=count_target/(count_target+count_loss)
    
    if (do_plots=='yes'):
        print('Target pct: {}'.format(target_pct))
        print('Stop Loss pct: {}'.format(stop_loss_pct))
        print('Total times in trade: {}'.format(count_trade))
        print('Number of gains: {}'.format(count_target))
        print('Number of losses: {}'.format(count_loss))
        print('% of trades that were gains: {:.2f}'.format(pct_trade_gain))
        print('Total gains:{:.2f}'.format(make_money))
        print('Total_losses: {:.2f}'.format(lose_money))
        print('net amount in the XXX scenerio: {:.2f}'.format(net_amount))
        print('-----------------------------------------------------------\n')
        #print('Ticker_data:\n', ticker_data.head(2))
        #print(ticker_data.columns)
        trade_plot(ticker,ticker_data,target_list,loss_list,trade_date_list,scenario)
    #store_trade_data(ticker,ticker_data.iloc[x])

In [24]:
def trade_on_conditions(ticker,ticker_data,target_pct,stop_loss_pct,num_shares,enum,do_plots):
    
    L=len(ticker_data.index)
    in_trade=0
    count_target=0
    count_loss=0
    count_trade=0
    make_money=0
    lose_money=0
    target_list=[]
    loss_list=[]
    trade_date_list=[]
    entry_price_list=[]
    exit_date_list=[]
    exit_price_list=[]
    ticker_data['Entry_Date']=np.nan
    ticker_data['Exit_Date']=np.nan
    ticker_data['Duration']=np.nan
    ticker_data['Target']=np.nan
    ticker_data['Stop_Loss']=np.nan
    ticker_data['GL']=np.nan
    ticker_data['GL_Status']=np.nan
    scenario='trade_on_conditions'
    
    A='Low'
    B='20ma'
    
    #  A=10ma and B=20ma cooresponds to TOT
    #  A=Close and B=10ma cooresponds to P10
    #  A=Close and B=20ma cooresponds to P20
    #  A=Close and B=10ma and C=20ma cooresponds to P10_P20
    #  A=low and B=20ma cooresponds to PA10_Low_PB20
    
    
    #print(L)
    for x in range(L):
        ma_check=ticker_data['Low'].iloc[x]-ticker_data['10ma'].iloc[x]
        daily_return_ma=ticker_data['Daily_Return_ma'].iloc[x]
        LL_Spread=ticker_data['LL_Spread'].iloc[x]
        Vol_pct=ticker_data['Vol_pct'].iloc[x]
        RSI=ticker_data['RSI'].iloc[x]
        Slope=ticker_data['Slope'].iloc[x]
        Slope_rate=ticker_data['Slope_rate'].iloc[x]
        
        #print(ma_check)
        #ma_check_previous= a['Close'].iloc[x-1]-a['10ma'].iloc[x-1]
        if (x==0):
            #continue
            ma_check_previous=ticker_data['Low'].iloc[0]-ticker_data['10ma'].iloc[0]
        else:
            ma_check_previous= ticker_data['Low'].iloc[x-1]-ticker_data['10ma'].iloc[x-1]
    

        #trigger condition that gets you in the trade-------------------------------------------------
        if (ma_check>0) and (ma_check_previous<0) and (daily_return_ma>0) and in_trade==0:
            in_trade=1
            count_trade=count_trade+1
            #print(ticker_data.index[x],'----',ticker_data['Close'].iloc[x])
            ticker_data.Stop_Loss.iloc[x]=ticker_data.Close.iloc[x]-(ticker_data.Close.iloc[x]*stop_loss_pct)
            #my_target=ticker_data['Target'].iloc[x]
            my_loss=ticker_data['Stop_Loss'].iloc[x]
            #the_gain=(my_target-ticker_data['Close'].iloc[x])*num_shares
            the_loss=(my_loss-ticker_data['Close'].iloc[x])*num_shares
            loss_list.append(ticker_data['Stop_Loss'].iloc[x])
            entry_price_list.append(ticker_data['Close'].iloc[x])
            trade_date_list.append(ticker_data.index[x])
            x_on_entry=x
            #ticker_data['Symbol'].iloc[x]=ticker
            #print(ticker_data['Target'].iloc[x],'---',ticker_data['Stop_Loss'].iloc[x],'---',count_trade)
            #print(ticker_data['High'].iloc[x],'---',ticker_data['Low'].iloc[x])
            continue
        
        #exit condition that gets you out the trade-------------------------------------------------
        if (in_trade==1):
            
            if (ticker_data['Close'].iloc[x]<ticker_data['10ma'].iloc[x]):
                #print('Target Reached:',ticker_data.index[x],'---',ticker_data['Target'].iloc[x_on_entry],'---',ticker_data['High'].iloc[x])
                the_gain=(ticker_data.Close.iloc[x]-ticker_data.Close.iloc[x_on_entry])*num_shares
                
                ticker_data.Entry_Date=ticker_data.index[x_on_entry].date()
                ticker_data.Exit_Date=ticker_data.index[x].date()
                #ticker_data.Duration=ticker_data.Exit_Date.iloc[x]-ticker_data.Entry_Date
                the_duration=ticker_data.Exit_Date.iloc[x]-ticker_data.Entry_Date
                ticker_data.Duration=the_duration.dt.days
                ticker_data.Target=ticker_data['10ma'].iloc[x]
                target_list.append(ticker_data['10ma'].iloc[x])
                #if (my_loss>=ticker_data['Close'].iloc[x]):
                        #exit_price_list.append(my_loss)
                        #exit_date_list.append(ticker_data.index[x])
                #else:
                    #exit_date_list.append(ticker_data.index[x])
                    #exit_price_list.append(ticker_data['Close'].iloc[x])
                    
                ticker_data.Stop_Loss=ticker_data.Close.iloc[x_on_entry]-(ticker_data.Close.iloc[x_on_entry]*stop_loss_pct)
                ticker_data.GL=the_gain
                
                if (the_gain>=0):
                    ticker_data.GL_Status=1
                    make_money=the_gain+make_money
                    count_target=count_target+1
                    exit_date_list.append(ticker_data.index[x])
                    exit_price_list.append(ticker_data['Close'].iloc[x])
                else:
                    ticker_data.GL_Status=0
                    count_loss=count_loss+1
                    if (my_loss>=ticker_data['Close'].iloc[x]):
                        exit_price_list.append(my_loss)
                        exit_date_list.append(ticker_data.index[x])
                        the_gain=(my_loss-ticker_data.Close.iloc[x_on_entry])*num_shares
                        ticker_data.GL=the_gain
                    else:
                        exit_date_list.append(ticker_data.index[x])
                        exit_price_list.append(ticker_data['Close'].iloc[x])
                    lose_money=the_gain+lose_money
                        
                store_trade_data(ticker,ticker_data,ticker_data.iloc[x_on_entry],scenario,enum)
                in_trade=0
            elif((my_loss>=ticker_data['Close'].iloc[x]) and (ticker_data[A].iloc[x]>ticker_data[B].iloc[x])):
                #print('Stop loss:',ticker_data.index[x],'---',ticker_data['Stop_Loss'].iloc[x_on_entry],'---',ticker_data['Low'].iloc[x])
                count_loss=count_loss+1
                lose_money=the_loss+lose_money
                ticker_data.Entry_Date=ticker_data.index[x_on_entry].date()
                ticker_data.Exit_Date=ticker_data.index[x].date()
                #ticker_data.Duration=ticker_data.Exit_Date.iloc[x]-ticker_data.Entry_Date
                the_duration=ticker_data.Exit_Date.iloc[x]-ticker_data.Entry_Date
                ticker_data.Duration=the_duration.dt.days
                ticker_data.Stop_Loss=ticker_data.Close.iloc[x_on_entry]-(ticker_data.Close.iloc[x_on_entry]*stop_loss_pct)
                ticker_data.Target.iloc[x]=my_loss
                target_list.append(my_loss)
                exit_price_list.append(my_loss)
                exit_date_list.append(ticker_data.index[x])
                ticker_data.GL=the_loss
                ticker_data.GL_Status=0
                store_trade_data(ticker,ticker_data,ticker_data.iloc[x_on_entry],scenario,enum)
                in_trade=0
                
    print(len(target_list),len(loss_list))
    print(len(trade_date_list),len(entry_price_list))
    print(len(exit_date_list),len(exit_price_list),'\n')
    
    if (len(entry_price_list)!=len(exit_price_list)):
        #del loss_list[-1]
        del trade_date_list[-1]
        del entry_price_list[-1]
        del target_list[-1]
        del loss_list[-1]
        count_trade=count_trade-1
        count_loss=count_loss-1
    
    print(len(target_list),len(loss_list))
    print(len(trade_date_list),len(entry_price_list))
    print(len(exit_date_list),len(exit_price_list))
        
        
        
    net_amount=make_money+lose_money
    pct_trade_gain=count_target/(count_target+count_loss)
    
    if (do_plots=='yes'):
        #print('Target pct: {}'.format(target_pct))
        print('Stop Loss pct: {}'.format(stop_loss_pct))
        print('Total times in trade: {}'.format(count_trade))
        print('Number of gains: {}'.format(count_target))
        print('Number of losses: {}'.format(count_loss))
        print('% of trades that were gains: {:.2f}'.format(pct_trade_gain))
        print('Total gains:{:.2f}'.format(make_money))
        print('Total_losses: {:.2f}'.format(lose_money))
        print('net amount in the trade on condition scenerio: {:.2f}'.format(net_amount))
        print('-----------------------------------------------------------\n')
        #print('Ticker_data:\n', ticker_data.head(2))
        #print(ticker_data.columns)
        print(target_list,len(target_list))
        print(loss_list,len(loss_list))
        print(trade_date_list,len(trade_date_list))
        print(entry_price_list,len(entry_price_list))
        print(exit_date_list,len(exit_date_list))
        print(exit_price_list,len(exit_price_list))
        
        trade_plot(ticker,ticker_data,target_list,loss_list,trade_date_list,scenario,entry_price_list,exit_price_list,exit_date_list)
    #store_trade_data(ticker,ticker_data.iloc[x])

In [25]:
def daily_scan(ticker,ticker_data,target_pct,stop_loss_pct,num_shares,enum,do_plots):
    
    L=len(ticker_data.index)
    in_trade=0
    count_target=0
    count_loss=0
    count_trade=0
    make_money=0
    lose_money=0
    target_list=[]
    loss_list=[]
    trade_date_list=[]
    entry_price_list=[]
    exit_date_list=[]
    exit_price_list=[]
    ticker_data['Entry_Date']=np.nan
    ticker_data['Exit_Date']=np.nan
    ticker_data['Duration']=np.nan
    ticker_data['two_c_ratio']=np.nan
    ticker_data['GL']=np.nan
    ticker_data['GL_Status']=np.nan
    
    scenario='daily_scan'
    
    day_delta=2
    #print(L)
    for x in range(L):
        if ((L-x)<=day_delta):
            break
        ma_check=ticker_data['Close'].iloc[x+day_delta]-ticker_data['Close'].iloc[x]
        daily_return_ma=ticker_data['Daily_Return_ma'].iloc[x]
        LL_Spread=ticker_data['LL_Spread'].iloc[x]
        Vol_pct=ticker_data['Vol_pct'].iloc[x]
        RSI=ticker_data['RSI'].iloc[x]
        Slope=ticker_data['Slope'].iloc[x]
        Slope_rate=ticker_data['Slope_rate'].iloc[x]
        neg_2sig_delta=ticker_data['Close'].iloc[x]-ticker_data['-2sigma'].iloc[x]
        ticker_data.two_c_ratio=(ticker_data['Close'].iloc[x]-ticker_data['-2sigma'].iloc[x])/ticker_data['Sigma_Spread'].iloc[x]
        entry_price_list.append(ticker_data['Close'].iloc[x])
        ticker_data.Entry_Date=ticker_data.index[x].date()
        ticker_data.Exit_Date=ticker_data.index[x+day_delta].date()
        the_duration=ticker_data.Exit_Date-ticker_data.Entry_Date
        ticker_data.Duration=the_duration.dt.days
                
        ticker_data.GL=ma_check
                
        if (ma_check>=0):
            ticker_data.GL_Status=1
        else:
            ticker_data.GL_Status=0
            
        #print('Date: {}_______GL Date{}'.format(ticker_data.index[x].date(),ticker_data.index[x+5].date()))
                               
        store_trade_data(ticker,ticker_data,ticker_data.iloc[x],scenario,enum)
        
    

In [26]:
#*********MAIN******************************

my_stock_list='low'
stocks=list_of_stocks(my_stock_list)
stocks=['AMZN','AES','AMD','CERN','GE','GM','NFLX','NRG','HPE','BSX']
print(stocks)
#stocks.sort()

start_date=dt.datetime(2013,1,1)
end_date=dt.date.today()
#my_target_pct=.1
#allowable_loss=.05
num_shares=100

the_target=[.05]
the_stop_loss=[.05]
enum=0
do_plots='no'
trade_scenario='daily_scan'
startTime=time.time()
for my_target_pct, allowable_loss in [(my_target_pct, allowable_loss) for my_target_pct in the_target for allowable_loss in the_stop_loss]:
    print(enum)
    for i in stocks:
        #print(i)
        ochlcv_data=get_stocks(i,start_date,end_date)
        indicators=calculate_indicators(i,ochlcv_data)
        #print(indicators.head(2))
        if (do_plots=='yes'):
            plot_the_stock(i,indicators,start_date)
            store_indicated_data(i,indicators)
        if (trade_scenario=='P10'):
            p10(i,indicators,my_target_pct,allowable_loss,num_shares,enum,do_plots)
        elif (trade_scenario=='TOT'):  
            TOT(i,indicators,my_target_pct,allowable_loss,num_shares,enum,do_plots)
        elif (trade_scenario=='low_sigma'):  
            low_sigma(i,indicators,my_target_pct,allowable_loss,num_shares,enum,do_plots)
        elif (trade_scenario=='XXX'):  
            XXX(i,indicators,my_target_pct,allowable_loss,num_shares,enum,do_plots)
        elif (trade_scenario=='trade_on_conditions'):  
            trade_on_conditions(i,indicators,my_target_pct,allowable_loss,num_shares,enum,do_plots)
        elif (trade_scenario=='daily_scan'):  
            daily_scan(i,indicators,my_target_pct,allowable_loss,num_shares,enum,do_plots)
        
    consolidated_summary(enum,my_target_pct,allowable_loss,trade_scenario)
    enum=enum+1
        
endTime=time.time()    
print('Duration :',endTime-startTime,' seconds')


['AMZN', 'AES', 'AMD', 'CERN', 'GE', 'GM', 'NFLX', 'NRG', 'HPE', 'BSX']
0
AMZN


C:\Users\Sharyn\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


AES
AMD
CERN
GE
GM
NFLX
NRG
HPE
BSX
Target pct: 0.05
Stop Loss pct: 0.05
Net G/L: 1787.8700000000067
G/L pct:
 1    0.527647
0    0.472353
Name: GL_Status, dtype: float64
-------------------------------------------------------------
Duration : 2457.0705964565277  seconds
